<a href="https://colab.research.google.com/github/patrickpato/Deep-Learning-Exercises/blob/main/Single_Entity_Sentiment_Analysis_3_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 4.3 MB/s 


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import WordPunctTokenizer
import nltk
#from nltk.sentiment.vader import SentimentIntensityAnalyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
import string
import re
nltk.download('words')
nltk.download('stopwords')
from gensim.models import KeyedVectors
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from wordcloud import STOPWORDS
from wordcloud import ImageColorGenerator
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from xgboost import XGBClassifier

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [51]:
#deep learning algos
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from tensorflow.keras import layers
from tensorflow import keras
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Conv1D,MaxPooling1D, Flatten, GRU
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

In [4]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
def load_data(path):
    df = pd.read_csv(path)
    return df
airtel_df = load_data('/content/gdrive/MyDrive/BrandComparison/airtel_labelled.csv')
saf_df = load_data('/content/gdrive/MyDrive/BrandComparison/safaricom_labelled.csv')

In [7]:
len(saf_df), len(saf_df.columns), len(airtel_df), len(airtel_df.columns)

(5258, 7, 2398, 7)

In [8]:
saf_df.head(2)

,date,tweet,clean_tweets,positivity_scores,negativity_scores,overall_scores,labels
0,2022-06-03 23:45:24+00:00,"@domoo_d Hi, apologies for the inconvenience c...",hi apologies inconvenience caused kindly fol...,0.265,0.147,0.2500,positive
1,2022-06-03 23:34:18+00:00,"@OlingaAbdul : Hello, We're currently experien...",hello we re currently experiencing system i...,0.300,0.000,0.8271,positive


In [9]:
le = LabelEncoder()
labels = saf_df['labels']
target = le.fit_transform(labels)

# Deep Learning Algos

## LSTM

In [10]:
#using keras' tokenizer for encoding. 
#we extract maximum features from the EDA on char length distribution
char_length = 60
def tokenize_text(max_char_length, text):
  max_features = max_char_length
  tokenizer = Tokenizer(num_words = max_features, split = " ")
  tokenizer.fit_on_texts(text)
  X = tokenizer.texts_to_sequences(text)
  X = pad_sequences(X)
  return X, len(tokenizer.word_index)


In [12]:
saf_df = saf_df.dropna()
airtel_df = airtel_df.dropna()

In [14]:
lstm_features_saf, vocab_size_saf = tokenize_text(60, saf_df['clean_tweets'])
lstm_features_saf.shape, vocab_size_saf

((5245, 40), 10115)

In [15]:
lstm_features_airtel, vocab_size_airtel = tokenize_text(60, airtel_df['clean_tweets'])
lstm_features_airtel.shape, vocab_size_airtel

((2394, 42), 4544)

In [30]:

embed_dim = 150
lstm_out = 196
lstm_model = Sequential()
lstm_model.add(Embedding(10115, embed_dim, input_length = 40)) # for airtel df, change from 10115 to 4544
lstm_model.add(SpatialDropout1D(0.2))
lstm_model.add(LSTM(lstm_out, dropout = 0.2, recurrent_dropout = 0.2))
lstm_model.add(Dense(3, activation="softmax"))
lstm_model.compile(loss="SparseCategoricalCrossentropy",optimizer="adam", metrics=["accuracy"])
print(lstm_model.summary())


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 40, 150)           1517250   
                                                                 
 spatial_dropout1d_4 (Spatia  (None, 40, 150)          0         
 lDropout1D)                                                     
                                                                 
 lstm_3 (LSTM)               (None, 196)               272048    
                                                                 
 dense_5 (Dense)             (None, 3)                 591       
                                                                 
Total params: 1,789,889
Trainable params: 1,789,889
Non-trainable params: 0
_________________________________________________________________
None


In [52]:

embed_dim = 150
GRU_out = 196
GRU_model = Sequential()
GRU_model.add(Embedding(10115, embed_dim, input_length = 40)) # for airtel df, change from 10115 to 4544
GRU_model.add(SpatialDropout1D(0.2))
GRU_model.add(GRU(GRU_out, dropout = 0.2, recurrent_dropout = 0.2))
GRU_model.add(Dense(3, activation="softmax"))
GRU_model.compile(loss="SparseCategoricalCrossentropy",optimizer="adam", metrics=["accuracy"])
print(GRU_model.summary())

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_18 (Embedding)    (None, 40, 150)           1517250   
                                                                 
 spatial_dropout1d_5 (Spatia  (None, 40, 150)          0         
 lDropout1D)                                                     
                                                                 
 gru_16 (GRU)                (None, 196)               204624    
                                                                 
 dense_12 (Dense)            (None, 3)                 591       
                                                                 
Total params: 1,722,465
Trainable params: 1,722,465
Non-trainable params: 0
_________________________________________________________________
None


In [57]:
#encoding categorical col
le = LabelEncoder()
saf_labels = le.fit_transform(saf_df['labels'])
airtel_labels = le.fit_transform(airtel_df['labels'])

In [36]:
tok_train, tok_test, y_train, y_test = train_test_split(lstm_features_saf, saf_labels)

In [37]:
#training
batch_size = 32
lstm_model.fit(tok_train, y_train, epochs = 20, batch_size=batch_size, verbose = 2)

Epoch 1/20
123/123 - 32s - loss: 0.9927 - accuracy: 0.5083 - 32s/epoch - 261ms/step
Epoch 2/20
123/123 - 30s - loss: 0.8989 - accuracy: 0.5601 - 30s/epoch - 248ms/step
Epoch 3/20
123/123 - 30s - loss: 0.8704 - accuracy: 0.5789 - 30s/epoch - 245ms/step
Epoch 4/20
123/123 - 30s - loss: 0.8617 - accuracy: 0.5823 - 30s/epoch - 244ms/step
Epoch 5/20
123/123 - 33s - loss: 0.8629 - accuracy: 0.5853 - 33s/epoch - 265ms/step
Epoch 6/20
123/123 - 30s - loss: 0.8571 - accuracy: 0.5945 - 30s/epoch - 245ms/step
Epoch 7/20
123/123 - 30s - loss: 0.8533 - accuracy: 0.5861 - 30s/epoch - 246ms/step
Epoch 8/20
123/123 - 30s - loss: 0.8432 - accuracy: 0.5985 - 30s/epoch - 247ms/step
Epoch 9/20
123/123 - 30s - loss: 0.8409 - accuracy: 0.5939 - 30s/epoch - 246ms/step
Epoch 10/20
123/123 - 32s - loss: 0.8364 - accuracy: 0.6046 - 32s/epoch - 260ms/step
Epoch 11/20
123/123 - 31s - loss: 0.8326 - accuracy: 0.6077 - 31s/epoch - 254ms/step
Epoch 12/20
123/123 - 31s - loss: 0.8269 - accuracy: 0.6087 - 31s/epoch - 

In [55]:
GRU_model.fit(tok_train, y_train, epochs = 20, batch_size=batch_size, verbose = 2)

Epoch 1/20
123/123 - 31s - loss: 0.9788 - accuracy: 0.5200 - 31s/epoch - 253ms/step
Epoch 2/20
123/123 - 28s - loss: 0.8951 - accuracy: 0.5627 - 28s/epoch - 226ms/step
Epoch 3/20
123/123 - 30s - loss: 0.8737 - accuracy: 0.5792 - 30s/epoch - 244ms/step
Epoch 4/20
123/123 - 28s - loss: 0.8650 - accuracy: 0.5858 - 28s/epoch - 228ms/step
Epoch 5/20
123/123 - 28s - loss: 0.8594 - accuracy: 0.5922 - 28s/epoch - 225ms/step
Epoch 6/20
123/123 - 28s - loss: 0.8540 - accuracy: 0.5965 - 28s/epoch - 230ms/step
Epoch 7/20
123/123 - 28s - loss: 0.8473 - accuracy: 0.5978 - 28s/epoch - 227ms/step
Epoch 8/20
123/123 - 27s - loss: 0.8403 - accuracy: 0.6069 - 27s/epoch - 222ms/step
Epoch 9/20
123/123 - 28s - loss: 0.8382 - accuracy: 0.6006 - 28s/epoch - 228ms/step
Epoch 10/20
123/123 - 29s - loss: 0.8354 - accuracy: 0.6051 - 29s/epoch - 237ms/step
Epoch 11/20
123/123 - 28s - loss: 0.8275 - accuracy: 0.6148 - 28s/epoch - 226ms/step
Epoch 12/20
123/123 - 28s - loss: 0.8256 - accuracy: 0.6095 - 28s/epoch - 

In [56]:
#testing model performance
lstm_score,lstm_acc = lstm_model.evaluate(tok_test, y_test, verbose = 2, batch_size = batch_size)
GRU_score, GRU_acc = GRU_model.evaluate(tok_test, y_test, verbose=2, batch_size=batch_size)
#print("score: %.2f" % (score))
print("LSTM acc: %.2f" % (lstm_acc))
print("GRU acc: %.2f" % (GRU_acc))

41/41 - 2s - loss: 0.8873 - accuracy: 0.5907 - 2s/epoch - 45ms/step
41/41 - 1s - loss: 0.8860 - accuracy: 0.6037 - 1s/epoch - 35ms/step
LSTM acc: 0.59
GRU acc: 0.60


## CNN

In [58]:
# create the model
cnn_model = Sequential()
cnn_model.add(Embedding(10115, embed_dim, input_length=char_length))
cnn_model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(256, activation='relu'))
cnn_model.add(Dense(3, activation='softmax'))
cnn_model.compile(loss='SparseCategoricalCrossentropy', optimizer='adam', metrics=['accuracy'])
cnn_model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 60, 150)           1517250   
                                                                 
 conv1d (Conv1D)             (None, 60, 128)           76928     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 30, 128)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 30, 64)            32832     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 15, 64)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 15, 32)          

In [59]:
# Fit the model
model.fit(tok_train, y_train, 
          validation_split=0.1,
          epochs=20, 
          batch_size=32, 
          verbose=1)

Epoch 1/20
111/111 [==============================] - 32s 255ms/step - loss: 0.9914 - accuracy: 0.5146 - val_loss: 0.9328 - val_accuracy: 0.5279
Epoch 2/20
111/111 [==============================] - 28s 252ms/step - loss: 0.8992 - accuracy: 0.5555 - val_loss: 0.8863 - val_accuracy: 0.5812
Epoch 3/20
111/111 [==============================] - 28s 254ms/step - loss: 0.8767 - accuracy: 0.5742 - val_loss: 0.8731 - val_accuracy: 0.5939
Epoch 4/20
111/111 [==============================] - 30s 267ms/step - loss: 0.8693 - accuracy: 0.5812 - val_loss: 0.8755 - val_accuracy: 0.5888
Epoch 5/20
111/111 [==============================] - 28s 252ms/step - loss: 0.8544 - accuracy: 0.5914 - val_loss: 0.8760 - val_accuracy: 0.5964
Epoch 6/20
111/111 [==============================] - 28s 253ms/step - loss: 0.8552 - accuracy: 0.5880 - val_loss: 0.8770 - val_accuracy: 0.5787
Epoch 7/20
111/111 [==============================] - 28s 250ms/step - loss: 0.8537 - accuracy: 0.5925 - val_loss: 0.9060 - val_ac

In [60]:
CNN_score, CNN_acc = model.evaluate(tok_test, y_test, verbose=2, batch_size=batch_size)
CNN_acc

41/41 - 2s - loss: 0.8667 - accuracy: 0.5907 - 2s/epoch - 39ms/step


0.5907012224197388